**Librerias**

In [1]:
import pandas as pd
import requests
from pathlib import Path

**Funciones**

In [19]:
def games_list():
    """Esta funcion contiene el listado de juegos a buscar """
    
    game_list = ['minecraft', 'call of duty', 'star wars']
    
    return game_list

In [20]:
def get_API_response(game_name):
    
    """Esta funcion obtiene la respuesta de la API al ingresar el nombre del videojuego"""
    
    url = "https://game-prices.p.rapidapi.com/games"

    querystring = {f"title":{game_name},
        "region":"us","offset":"0","limit":"100"}

    headers = {
        "X-RapidAPI-Key": "",
        "X-RapidAPI-Host": "game-prices.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return response

In [21]:
def json_cleaning(response):
    
    """Esta funcion realiza la limpieza de la data y crea un dataframe con los precios del videojuego"""
    
    res_json = response.json()
    df = pd.DataFrame(res_json)
    df_game = pd.json_normalize(df['games'])
    exploded = df_game.explode('stores')
    df_final = pd.concat([exploded
               [["id","name","releaseDate","type","currentLowestPrice","currency","availability"]]
               .reset_index(drop=True), pd.json_normalize(exploded['stores'])], axis=1)
    
    return df_final

In [22]:
def game_prices():
    
    """Esta funcion crea un archivo y lo coloca dentro de un folder especifico a traves de la obtencion 
    y procesamiento de la informacion obtenida de la API """
    
    url = "https://game-prices.p.rapidapi.com/games"
    headers = {
        "X-RapidAPI-Key": "fedb574b3fmsh0e54c4b50515cd9p18c3d3jsn15b32ccc2828",
        "X-RapidAPI-Host": "game-prices.p.rapidapi.com"
    }
    game_list = games_list()
    
    for num, game in enumerate(game_list, start=1):

        print(f"This game is #{num}")
        filename = f"{game} game prices.csv"
        filename = "data_game/"+ filename
        print("Document will be place and call: "+ filename)

        if not Path(filename).is_file():
            page_source = get_API_response(game)
            print(game)
            print("Finish extracting game prices.")
            print("cleanning content.")
            df = json_cleaning(page_source)
            print("File Building and Appending.")

            df.to_csv(filename, index = True)
            print(f"{filename} saved.")
            print("--------"*5)
            print()

    print("Finished!")
    
    

**Prueba**

In [18]:
game_prices()

This game is #1
Document will be place and call: data_game/minecraft game prices.csv
minecraft
Finish extracting game prices.
cleanning content.
File Building and Appending.
data_game/minecraft game prices.csv saved.
----------------------------------------

This game is #2
Document will be place and call: data_game/call of duty game prices.csv
call of duty
Finish extracting game prices.
cleanning content.
File Building and Appending.
data_game/call of duty game prices.csv saved.
----------------------------------------

This game is #3
Document will be place and call: data_game/star wars game prices.csv
star wars
Finish extracting game prices.
cleanning content.
File Building and Appending.
data_game/star wars game prices.csv saved.
----------------------------------------

Finished!
